# Variants, lineages, and mutations set creation
Variants, lineages, and mutations are of interest but are not necessarily confined to any specific topicCategory. Classifying for these might help to search for VOCs and VOIs.

User regex to find specific lineages, or mutations

Regex cheatcode:
* '\b' indicates word boundary
* '()' indicates a capture group
* '?:' indicates a non-capture group
* '\d' indicates a digit
* '{1,5}' indicates repeat in pattern between 1 and 5 times (so in mutation example, one to five digits)
* "r' " indicates raw string notation
* (?i)(?-i) indicates everything between is case insensitive

In [ ]:
import os
import requests
import json
import pandas as pd
from pandas import read_csv
from datetime import datetime


In [ ]:
DATAPATH = 'data/'
RESULTSPATH = 'results/'

In [ ]:

def get_ids_from_json(jsonfile):
    idlist = []
    for eachhit in jsonfile["hits"]:
        if eachhit["_id"] not in idlist:
            idlist.append(eachhit["_id"])
    return(idlist)



#### Get the size of the source (to make it easy to figure out when to stop scrolling)
def fetch_src_size(source):
    pubmeta = requests.get("https://api.outbreak.info/resources/query?q=((@type:Publication) AND (curatedBy.name:"+source+"))&size=0&aggs=@type")
    pubjson = json.loads(pubmeta.text)
    pubcount = int(pubjson["facets"]["@type"]["total"])
    return(pubcount)


#### Ping the API and get all the ids for a specific source and scroll through the source until number of ids matches meta
def get_source_ids(source):
    source_size = fetch_src_size(source)
    r = requests.get("https://api.outbreak.info/resources/query?q=((@type:Publication) AND (curatedBy.name:"+source+"))&fields=_id&fetch_all=true")
    response = json.loads(r.text)
    idlist = get_ids_from_json(response)
    try:
        scroll_id = response["_scroll_id"]
        while len(idlist) < source_size:
            r2 = requests.get("https://api.outbreak.info/resources/query?q=((@type:Publication) AND (curatedBy.name:"+source+"))&fields=_id&fetch_all=true&scroll_id="+scroll_id)
            response2 = json.loads(r2.text)
            idlist2 = set(get_ids_from_json(response2))
            tmpset = set(idlist)
            idlist = tmpset.union(idlist2)
            try:
                scroll_id = response2["_scroll_id"]
            except:
                print("no new scroll id")
        return(idlist)
    except:
        return(idlist)


def get_pub_ids(sourceset):
    pub_srcs = {"preprint":["bioRxiv","medRxiv"],"litcovid":["litcovid"],
                "other":["Figshare","Zenodo","MRC Centre for Global Infectious Disease Analysis"],
                "all":["Figshare","Zenodo","MRC Centre for Global Infectious Disease Analysis",
                       "bioRxiv","medRxiv","litcovid"]}
    sourcelist = pub_srcs[sourceset]
    allids = []
    for eachsource in sourcelist:
        sourceids = get_source_ids(eachsource)
        allids = list(set(allids).union(set(sourceids)))
    return(allids)




In [ ]:
starttime = datetime.now()
allids = get_pub_ids('all')
print('fetched all ids: ',datetime.now()-starttime)
starttime = datetime.now()
metadf = batch_fetch_dated_meta(allids)
print('fetched all metadata: ',datetime.now()-starttime)
starttime = datetime.now()
textdf = dirty_merge_texts(metadf)
print('merged all text: ',datetime.now()-starttime)
print(textdf.head(n=2))

In [ ]:
textdf.to_csv('data/textdf.txt',sep='\t',header=True)

In [ ]:
textdf = read_csv('data/textdf.txt',delimiter='\t',header=0,index_col=0)

### To do
1. Find and extract mutations
2. Check frequency of mutations in publications to see if trends can be identified
3. Train algorithms to see if any new true positives can be identified
3. Do the same for lineages

In [ ]:
#### Mutation training set creation
import re

genes = "(?:ORF1a|ORF1b|S|ORF3a|ORF3b|E|M|ORF6|ORF7a|ORF7b|ORF8|N|ORF9b|ORF10|ORF14|3'UTR|3UTR)"
#proteins = "S|E|N|M|(NSP|Nsp(?:1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16))"
geneprots = r"\b(?:ORF1a|ORF1b|Spike|spike|ORF3a|ORF3b|Envelope|envelope|M protein|M\(pro\)|ORF6|ORF7a|ORF7b|ORF8|ORF9b|ORF10|ORF14|3'UTR|3UTR|(?:(?:NSP|nsp|Nsp|N)(?:1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16)))\b"
mutations = "((?:A|C|D|E|F|G|H|I|K|L|M|N|P|Q|R|S|T|V|W|Y)\d{1,5}(?:A|C|D|E|F|G|H|I|K|L|M|N|P|Q|R|S|T|V|W|Y))"
#mutations = "^(?:A|C|D|E|F|G|H|I|K|L|M|N|P|Q|R|S|T|V|W|Y)\d{1,5}(?:A|C|D|E|F|G|H|I|K|L|M|N|P|Q|R|S|T|V|W|Y)$"

#deletions = genes+":"+"(?:DEL|Del|del)"+"\d{}"
#deletion_ex = "ORF8∆381, ORF7a∆81 and spike∆15, ∆15, 60/70-deletion"
#deletion_variant = r"\b(∆\d{1,5} variant)\b"

token_dict = {
    'mutants':r'\b((?:A|C|D|E|F|G|H|I|K|L|M|N|P|Q|R|S|T|V|W|Y)\d{2,5}(?:A|C|D|E|F|G|H|I|K|L|M|N|P|Q|R|S|T|V|W|Y))\b',
    #'genemute':r"\b((?:ORF1a|ORF1b|S|ORF3a|ORF3b|E|M|ORF6|ORF7a|ORF7b|ORF8|N|ORF9b|ORF10|ORF14|3'UTR|3UTR):(?:A|C|D|E|F|G|H|I|K|L|M|N|P|Q|R|S|T|V|W|Y)\d{1,5}(?:A|C|D|E|F|G|H|I|K|L|M|N|P|Q|R|S|T|V|W|Y))\b",
    'genemute':r"\b((?:ORF1a|ORF1b|S|Spike|spike|ORF3a|ORF3b|E|Envelope|envelope|M|M protein|M\(pro\)|ORF6|ORF7a|ORF7b|ORF8|ORF9b|ORF10|ORF14|3'UTR|3UTR|(?:(?:NSP|nsp|Nsp|N)(?:1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16)))(?:\s|:)(?:A|C|D|E|F|G|H|I|K|L|M|N|P|Q|R|S|T|V|W|Y)\d{1,5}(?:A|C|D|E|F|G|H|I|K|L|M|N|P|Q|R|S|T|V|W|Y))\b",
    'deletions':r"\b((?:ORF1a|ORF1b|S|Spike|spike|ORF3a|ORF3b|E|Envelope|envelope|M|M protein|M\(pro\)|ORF6|ORF7a|ORF7b|ORF8|ORF9b|ORF10|ORF14|3'UTR|3UTR|(?:(?:NSP|nsp|Nsp|N)(?:1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16)))(?:∆|(?:DEL|Del|del|:DEL|:Del|:del))\d{1,5})\b",
    'nonspec_deletion':r"\b((?:ORF1a|ORF1b|S|Spike|spike|ORF3a|ORF3b|E|Envelope|envelope|M|M protein|M\(pro\)|ORF6|ORF7a|ORF7b|ORF8|N|ORF9b|ORF10|ORF14|3'UTR|3UTR|(?:NSP|Nsp|nsp)(?:1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16))\s(?:(?:\d{1,5})(?:\/)(?:\d{1,5})|(?:\d{1,5}))(?:\s|-)(?:deletion))\b"
}

testtext = "This gene is full of ORF1a∆123 variants or other kinds of NSP deletion variants. There are also various mutations like S:E484K which is also known just as E484K which is a spike gene mutation. There are also deletions on the spike protein such as S:DEL15 or S∆15 though it could also be S:del15 so to speak or spike:del15 or Sdel15. Imagine if there was an ORF1a:A645T or E:C163G mutation. There are also nonspecific deletions like 60/70-deletion or 145/162-deletion or ∆15. There can also be mutations or deletions in the NSP genes or proteins such as Nsp2∆115 or NSP1:del115. Is it b.1.91 going to work or will any sort of lineageb.1.91 work nope and the spike 60/70-deletion or maybe the NSP2 459 deletion"

In [ ]:
mutationslist = pd.DataFrame(columns=['_id','name','abstract','description','text','date','mutations'])
for eachkey in token_dict.keys():
    tmpdf = textdf.loc[textdf['text'].str.contains(token_dict[eachkey])].copy()
    tmpdf['mutations'] = tmpdf['text'].str.findall(token_dict[eachkey])
    tmpmutationslist = tmpdf.explode('mutations').copy()
    mutationslist = pd.concat((mutationslist,tmpmutationslist),ignore_index=True)
mutationslist['date'] = pd.to_datetime(mutationslist['date'])
mutationslist.drop_duplicates(keep='first',inplace=True)
mutationslist['gene_mentions'] = mutationslist['text'].str.findall(geneprots)
mutationslist['gene_mentions'] = mutationslist['gene_mentions'].apply(lambda x: lowerlist(x))
print(mutationslist.head(n=2))

In [ ]:
mutationsclean = mutationslist[['_id','name','date','mutations','gene_mentions']].copy()
mutationsclean.to_csv(os.path.join(RESULTSPATH,'mutations.tsv'),sep='\t',header=True)

In [ ]:
humefactors = mutationslist.loc[mutationslist['text'].str.contains("polymorphism")].copy()
humefactors.drop_duplicates(keep="first",inplace=True)
humefactors.drop(columns=['abstract','text','description'],inplace=True)
humefactors.to_csv(os.path.join(RESULTSPATH,'polymorphisms.tsv'),sep='\t',header=True)

In [ ]:
print(len(textdf))
#mutdf = textdf.loc[(textdf['text'].str.contains(token_dict['mutants']))].copy()
#mutdf['mutations'] = mutdf['text'].str.findall(token_dict['mutants'])

#mutdf['date'] = pd.to_datetime(mutdf['date'])
#mutationslist = mutdf.explode('mutations').copy()
#mutationslist.drop(['abstract','name','description'],axis=1,inplace=True)
#sortedmutations = mutationslist.sort_values(['mutations','date'],ascending=[True,True])
#mutationfrequency = mutationslist.groupby('mutations').size().reset_index(name='counts')
mutationfrequency = mutationslist.groupby('mutations').resample('W-Mon', on='date').size().reset_index(name='counts').sort_values(by='date')
mutationfrequency.sort_values(['date','counts'],ascending=[False,False],inplace=True)
print(mutationslist.head(n=5))
print(mutationfrequency.head(n=20))
#singlementions = mutationfrequency['mutations'].loc[mutationfrequency['counts']<2].unique().tolist()
#mutations2check = mutationslist.loc[mutationslist['mutations'].isin(singlementions)]
#mutations2check.to_csv('data/variants/mutations_to_check.tsv',sep='\t',header=True)

In [ ]:
#genemutdf = textdf.loc[textdf['text'].str.contains(token_dict['genemute'],case=False)].copy()
#genemutdf = textdf.loc[textdf['text'].str.extract(token_dict['genemute'])].copy()
genemutdf = textdf.loc[textdf['text'].str.contains('E484K')].copy()
print(genemutdf)

In [ ]:
#### Variant training set creation


"""
Variant of Concern 202012/01", "VOC-202012/01", "20B/501Y.V1", "20I/501Y.V1", the 501Y.V2 variant
Variant Under Investigation 202012/01 (VUI 202012/01 for short)
"the {location} variant"
"variant in {location}"
"""

the_variant = r"((?i)the (?:\w*|\w*.\w*) variant(?-i))"


In [ ]:
#### Lineage training set creation
## More lineage names: "https://www.the-scientist.com/news-opinion/a-guide-to-emerging-sars-cov-2-variants-68387"

## fetch lineages from :"https://cov-lineages.org/lineages.html"
lineagetable = read_csv("https://raw.githubusercontent.com/cov-lineages/pango-designation/master/lineages.csv",error_bad_lines=False,header=0)
lineages = lineagetable['lineage'].loc[lineagetable['lineage'].str.len()>2].unique().tolist()
print(lineagetable,len(lineages))

In [ ]:
wikivariants = variant_names(querylist)
wikidict = {}
i=0
while i < len(wikivariants):
    wikidict[wikivariants.iloc[i]['alias'].lower().strip()] = wikivariants.iloc[i]['name'].lower().strip()
    i=i+1

## Lineage search, no additional regex formatting
The issue with this method is that the (.) are not handled properly so things like N95 end up being included and an additional filtering is needed which may end up cutting out relevant entries

In [ ]:
masterlist = list(set(lineages).union(set(wikivariants['alias'].tolist())))
searchterm = ' | '.join(masterlist)
filterterms = '|'.join(filter_terms)
tmpdf = textdf.loc[textdf['text'].str.contains(searchterm)].copy()
tmpdf['lineages'] = tmpdf['text'].str.findall(searchterm)
rawlineageslist = tmpdf.explode('lineages').copy()
cleanlineageslist = rawlineageslist.loc[rawlineageslist['text'].str.contains(filterterms)].copy()
cleanlineageslist['lineages'] = [x.strip() for x in cleanlineageslist['lineages']]

print(len(rawlineageslist))
print(len(cleanlineageslist))
    
cleanlineageslist['lineages'].replace(wikidict,inplace=True)
cleanlineageslist.drop_duplicates(keep='first',inplace=True)
print(len(cleanlineageslist))
print(cleanlineageslist.head(n=10))

In [ ]:
lineagecounts = cleanlineageslist.groupby('lineages').size().reset_index(name='publication counts')
lineagecounts.sort_values('publication counts',ascending=False,inplace=True)
print(lineagecounts)

## Additional regex attempts

In [ ]:
masterlist = list(set(lineages).union(set(wikivariants['alias'].tolist())))
#print(len(masterlist))
searchterm = ' | '.join(masterlist)
#re_str1 = r'\b(?i)('
#re_str2 = r')(?-i)'
#rawstring = r"{}".format(searchterm)
#searchregex = re.compile(re_str1 + rawstring + re_str2)
filterterms = '|'.join(filter_terms)

In [ ]:
masterlist = list(set(lineages).union(set(wikivariants['alias'].tolist())))
regexlist = []
for eachitem in masterlist:
    #rawstring = r"{}".format(eachitem.strip().replace('.','\.'))
    searchstring = rf"{re.escape(eachitem)}"
    #searchregex = re.compile(searchstring)
    #print(searchregex)
    regexlist.append(searchstring)
    #checkdf = textdf.loc[textdf['text'].str.contains(searchregex)].copy()
    #if len(checkdf)>0:
    #    checkdf['lineages'] = checkdf['text'].str.findall(searchregex)
    #    print(checkdf)
    

In [ ]:
#searchterms = '|'.join(regexlist)
searchregex = re.compile('|'.join(regexlist), re.IGNORECASE)
lineagedf = textdf.loc[textdf['text'].str.contains(searchregex)].copy()
lineagedf['lineages'] = lineagedf['text'].str.findall(searchregex)
print(len(lineagedf))

In [ ]:
rawlineageslist = lineagedf.explode('lineages').copy()
#cleanlineageslist = rawlineageslist.loc[rawlineageslist['text'].str.contains(filterterms)].copy()
cleanlineageslist = rawlineageslist[['_id','name','date','lineages']].copy()
cleanlineageslist['lineages'] = [x.strip().lower() for x in cleanlineageslist['lineages']]

print(len(rawlineageslist))
print(len(cleanlineageslist))
    
cleanlineageslist['lineages'].replace(wikidict,inplace=True)
cleanlineageslist.drop_duplicates(keep='first',inplace=True)
print(len(cleanlineageslist))
print(cleanlineageslist.head(n=10))
cleanlineageslist.to_csv('results/lineages.tsv',sep='\t',header=True)

In [ ]:
frequency = cleanlineageslist.groupby('lineages').size().reset_index(name='counts')
frequency.sort_values('counts',ascending=False,inplace=True)
print(frequency.head(n=10))
print(frequency.tail(n=10))

### Check use of elasticsearch to correctly query for ambiguous lineages
A development API has been created to test the effect of ignoring '.' in the indexing of name, abstract, keywords

In [ ]:
def fetch_dev_size(source):
    pubmeta = requests.get("https://dev.outbreak.info/resources/query?q=((@type:Publication) AND (curatedBy.name:"+source+"))&size=0&aggs=@type")
    pubjson = json.loads(pubmeta.text)
    pubcount = int(pubjson["facets"]["@type"]["total"])
    return(pubcount)


#### Ping the API and get all the ids for a specific source and scroll through the source until number of ids matches meta
def get_dev_ids(source):
    source_size = fetch_src_size(source)
    r = requests.get("https://dev.outbreak.info/resources/query?q=((@type:Publication) AND (curatedBy.name:"+source+"))&fields=_id&fetch_all=true")
    response = json.loads(r.text)
    idlist = get_ids_from_json(response)
    try:
        scroll_id = response["_scroll_id"]
        while len(idlist) < source_size:
            r2 = requests.get("https://dev.outbreak.info/resources/query?q=((@type:Publication) AND (curatedBy.name:"+source+"))&fields=_id&fetch_all=true&scroll_id="+scroll_id)
            response2 = json.loads(r2.text)
            idlist2 = set(get_ids_from_json(response2))
            tmpset = set(idlist)
            idlist = tmpset.union(idlist2)
            try:
                scroll_id = response2["_scroll_id"]
            except:
                print("no new scroll id")
        return(idlist)
    except:
        return(idlist)

## Cleaning up the code

In [ ]:
%%time
## Extract mutations
import os
import re
import requests
import json
import pandas as pd
from pandas import read_csv
from datetime import datetime
from src.common import *
from src.extract_variants import *

DATAPATH = 'data/'
RESULTSPATH = 'results/'


allids = get_pub_ids('all')
metadf = batch_fetch_meta(allids)
textdf = merge_texts(metadf)
mutationsclean = extract_mutations(RESULTSPATH,textdf, token_dict, export=True)

## Run time of entire script: 31 min
## Run time of just the mutation extraction: 24.7 s

In [ ]:
%%time
## Extract variants
import os
import re
import requests
import json
import pandas as pd
import pickle
from pandas import read_csv
from datetime import datetime
from src.common import *
from src.extract_variants import *

DATAPATH = 'data/'
RESULTSPATH = 'results/'

#allids = get_pub_ids('all')
#metadf = batch_fetch_meta(allids)
#textdf = merge_texts(metadf)
cleanlineageslist = extract_lineages(DATAPATH,RESULTSPATH,lineagequerylist,textdf,export=True)


## Runtime: 
## Runtime of just the lineage extraction: 1 hr 5 min

In [ ]:
from pandas import read_csv
textdf = read_csv('data/textdf.txt',index_col=0,header=0,delimiter='\t')
print(textdf.head(n=2))

In [5]:
%%time
import os
import re
import requests
import json
import pathlib
import pandas as pd
from pandas import read_csv
from datetime import datetime
from src.common import *
from src.extract_variants import *

#script_path = pathlib.Path(__file__).parent.absolute()
script_path = ''
DATAPATH = os.path.join(script_path,'data/')
RESULTSPATH = os.path.join(script_path,'results/')

allids = get_pub_ids('all')
metadf = batch_fetch_dated_meta(allids)
textdf = merge_texts(metadf)
mutationsclean = extract_mutations(RESULTSPATH, textdf, token_dict, export=True)
cleanlineageslist = extract_lineages(DATAPATH, RESULTSPATH, lineagequerylist, textdf, export=True)

## Run time 1h 42 min

C:\Users\gtsueng\Anaconda3\envs\outbreak\lib\site-packages\pandas\core\strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


Wall time: 1h 42min 34s
